In [1]:
# the goal here is to make a few-shot prompt where all the answers are A, while the answer for the target question is B
# then the model is in some sense being "deceptive" when picking A

import torch
import numpy as np
import torch
import random

seed = 633
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed);

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer
from datasets import load_dataset, Dataset

ds_name = "sciq"
n_total = 3000
orig_ds: Dataset = load_dataset(ds_name, split="train").shuffle(seed=seed).select(range(n_total))
orig_ds


/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset sciq (/home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493)
Loading cached shuffled indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-8e3ed5fb4e584f24.arrow


Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 3000
})

In [3]:
def get_text_example(example, a_correct=True, few_shot_prefix=None):
    """a_correct: whether the correct answer is A or B"""
    # get the true answer and one distractor
    dist_name = random.choice(["distractor1", "distractor2", "distractor3"])
    distractor = example[dist_name]
    q = example["question"]
    ans = example["correct_answer"]
    support = example["support"]
    prefix = few_shot_prefix or ""

    option_a, option_b = (ans, distractor) if a_correct else (distractor, ans)
    prompt = f"{prefix}{support}\n\nQ: {q}\n  (A) {option_a}\n  (B) {option_b}\n\nA:"
    label = "A" if a_correct else "B"
    return prompt, label

def make_few_shot_prompt(few_shot_exs, always_a_correct=False):
    """make a few-shot prompt
    always_a_correct: whether the correct answer is always A or is randomly A or B
    """

    make_example = lambda prompt, label: f"{prompt} {label}\n\n"
    a_correct = lambda: True if always_a_correct else random.choice([True, False])
    examples = [make_example(*get_text_example(example, a_correct=a_correct())) for example in few_shot_exs]
    return "".join(examples)

In [4]:
split_ds = orig_ds.train_test_split(test_size=0.9, seed=seed)
few_shot_ds, split_ds = split_ds["train"], split_ds["test"]
split_ds = split_ds.train_test_split(test_size=0.5, seed=seed)
source_ds, decept_ds = split_ds["train"], split_ds["test"]
len(source_ds), len(decept_ds), len(few_shot_ds)

Loading cached split indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-e80177cfb879e008.arrow and /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-08ef0965c6b04529.arrow
Loading cached split indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-60cfcd3139a28e95.arrow and /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-4612fb9c74290b98.arrow


(1350, 1350, 300)

In [5]:
from utils import load_model_and_tokenizer, call_model

model_name = "gpt2-xl"
is_llama = "llama" in model_name or "vicuna" in model_name
model, tokenizer = load_model_and_tokenizer(model_name, is_llama)

In [6]:
num_shots = 6
n_layer = model.config.num_hidden_layers
hidden_size = model.config.hidden_size
    

In [15]:
from tqdm import tqdm

ds = source_ds
def get_hiddens(ds, randomize):
    lm_p_corrects = np.zeros(len(ds))
    hiddens = np.zeros((len(ds), n_layer + 1, hidden_size))
    texts = np.empty(len(ds), dtype=object)
    labels = np.empty(len(ds), dtype=object)
    for i, example in enumerate(tqdm(decept_ds, total=len(ds))):
        few_shot_examples = few_shot_ds.shuffle(seed=seed).select(range(num_shots))
        few_shot_prompt = make_few_shot_prompt(few_shot_examples, always_a_correct=not randomize)

        a_correct = random.choice([True, False]) if randomize else False
        prompt, label = get_text_example(example, a_correct=a_correct, few_shot_prefix=few_shot_prompt)
        hidden_states, logits = call_model(model, tokenizer, prompt, raise_too_long=False)

        target_tokens = ["A", "B"] if is_llama else [" A", " B"]
        a_id, b_id = tokenizer(target_tokens, return_tensors="pt", add_special_tokens=False).input_ids.squeeze(1)
        p_a, p_b = logits[0, -1, [a_id, b_id]].softmax(dim=-1).cpu().numpy()


        hiddens[i, :, :] = torch.cat(hidden_states).cpu().numpy()[:, -1]  # all layers, last token
        lm_p_corrects[i] = p_b  # B is the correct answer
        texts[i] = prompt
        labels[i] = label

    return hiddens, lm_p_corrects, texts, labels

source_hiddens, source_lm_p_corrects, source_texts, source_labels = get_hiddens(source_ds, randomize=True)


  0%|          | 0/1350 [00:00<?, ?it/s]Loading cached shuffled indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-d6adc9dc7b98aac5.arrow
Loading cached shuffled indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-d6adc9dc7b98aac5.arrow
  0%|          | 2/1350 [00:00<01:37, 13.87it/s]Loading cached shuffled indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-d6adc9dc7b98aac5.arrow
Loading cached shuffled indices for dataset at /home/alex/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493/cache-d6adc9dc7b98aac5.arrow
  0%|          | 4/1350 [00:00<01:30, 14.88it/s]Loading cached shuffled indices for dataset at /home/alex/.cache/hug

In [8]:
decept_hiddens, decept_lm_p_corrects, decept_texts, decept_labels = get_hiddens(decept_ds, randomize=False)

Hi


In [22]:
# save results
from pathlib import Path
import time

prefix = time.strftime("%Y-%m-%d-%H:%M")
cache_dir = Path("./lr-experiments") / ds_name / prefix
cache_dir.mkdir(exist_ok=True, parents=True)
np.save(cache_dir / "source_hiddens.npy", source_hiddens)
np.save(cache_dir / "source_lm_p_corrects.npy", source_lm_p_corrects)
np.save(cache_dir / "source_texts.npy", source_texts)
np.save(cache_dir / "source_labels.npy", source_labels)
np.save(cache_dir / "decept_hiddens.npy", decept_hiddens)
np.save(cache_dir / "decept_lm_p_corrects.npy", decept_lm_p_corrects)
np.save(cache_dir / "decept_texts.npy", decept_texts)
np.save(cache_dir / "decept_labels.npy", decept_labels)

In [26]:
# split source into train and val using sklearn
from sklearn.model_selection import train_test_split
train_hiddens, val_hiddens, train_lm_p_corrects, val_lm_p_corrects, train_texts, val_texts, train_labels, val_labels = train_test_split(
    source_hiddens, source_lm_p_corrects, source_texts, source_labels, test_size=0.3, random_state=seed
)

In [27]:
# train a LR probe on the train set
from sklearn.linear_model import LogisticRegressionCV

Cs = 10 ** np.linspace(-5, 5, 11)
n_layer = model.config.num_hidden_layers
layer = 25  # 29 * n_layer // 30 + 1  # the layer to use for classification, somewhat arbitrary but middle layers work better
reporter = LogisticRegressionCV(Cs=Cs, cv=2).fit(train_hiddens[:, layer, :], train_labels)

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [29]:
train_preds = reporter.predict(source_hiddens[:, layer, :])
train_acc = (train_preds == source_labels).mean()
train_sem = np.std(train_preds == source_labels) / np.sqrt(len(train_preds))
print(f"reporter train acc: {train_acc:.3f} ± {2 * train_sem:.3f}")

val_preds = reporter.predict(val_hiddens[:, layer, :])
val_acc = (val_preds == val_labels).mean()
val_sem = np.std(val_preds == val_labels) / np.sqrt(len(val_preds))
print(f"reporter val acc: {val_acc:.3f} ± {2 * val_sem:.3f}")

decept_preds = reporter.predict(decept_hiddens[:, layer, :])
decept_acc = (decept_preds == decept_labels).mean()
decept_sem = np.std(decept_preds == decept_labels) / np.sqrt(len(decept_preds))
print(f"reporter acc with leading prefix: {decept_acc:.3f} ± {2 * decept_sem:.3f}")

reporter train acc: 0.953 ± 0.012
reporter val acc: 0.842 ± 0.036
reporter acc with leading prefix: 0.838 ± 0.020


In [32]:
# now for LM probs

val_lm_is_correct = val_lm_p_corrects > 0.5
val_lm_acc = (val_lm_is_correct).mean()
val_lm_sem = np.std(val_lm_is_correct) / np.sqrt(len(val_lm_is_correct))
print(f"LM val acc: {val_lm_acc:.3f} ± {2 * val_lm_sem:.3f}")

decept_lm_is_correct = decept_lm_p_corrects > 0.5
decept_lm_acc = (decept_lm_is_correct).mean()
decept_lm_sem = np.std(decept_lm_is_correct) / np.sqrt(len(decept_lm_is_correct))
print(f"LM acc with leading prefix: {decept_lm_acc:.3f} ± {2 * decept_lm_sem:.3f}")


LM val acc: 0.923 ± 0.026
LM acc with leading prefix: 0.361 ± 0.026


In [ ]:
# get a LM to generate rationalizations for arbitrary statements
# first, we need to generate some good CoT responses to use as few-shot training data
# then we write a few-shot prompt where all the answers are A, but in the final example the answer is B